In [1]:
import torch
import os
from torch.autograd import Variable
from torchvision import models

MODEL_DIR = "outputs/onnx_test"
#vgg16 = torch.load(os.path.join(MODEL_DIR, "model.pt"))
vgg16 = models.vgg16(pretrained=True)
vgg16 = vgg16.cuda()

In [2]:
from models import DisentangledLinear
import torch.nn as nn
from collections import namedtuple

class Vgg16Experimental(torch.nn.Module):
    
    def __init__(self, vgg16, requires_grad=False, show_progress=False):
        super().__init__()
        vgg_pretrained_features = vgg16.features

        # I've exposed the best/most interesting layers in my subjective opinion (mp5 is not that good though)
        self.layer_names = ['relu3_3', 'relu4_1', 'relu4_2', 'relu4_3', 'relu5_1', 'relu5_2', 'relu5_3', 'mp5', 'ap', "relu_cl1", "relu_cl2"]

        # 31 layers in total for the VGG16
        self.conv1_1 = vgg_pretrained_features[0]
        self.relu1_1 = vgg_pretrained_features[1]
        self.conv1_2 = vgg_pretrained_features[2]
        self.relu1_2 = vgg_pretrained_features[3]
        self.max_pooling1 = vgg_pretrained_features[4]
        self.conv2_1 = vgg_pretrained_features[5]
        self.relu2_1 = vgg_pretrained_features[6]
        self.conv2_2 = vgg_pretrained_features[7]
        self.relu2_2 = vgg_pretrained_features[8]
        self.max_pooling2 = vgg_pretrained_features[9]
        self.conv3_1 = vgg_pretrained_features[10]
        self.relu3_1 = vgg_pretrained_features[11]
        self.conv3_2 = vgg_pretrained_features[12]
        self.relu3_2 = vgg_pretrained_features[13]
        self.conv3_3 = vgg_pretrained_features[14]
        self.relu3_3 = vgg_pretrained_features[15]
        self.max_pooling3 = vgg_pretrained_features[16]
        self.conv4_1 = vgg_pretrained_features[17]
        self.relu4_1 = vgg_pretrained_features[18]
        self.conv4_2 = vgg_pretrained_features[19]
        self.relu4_2 = vgg_pretrained_features[20]
        self.conv4_3 = vgg_pretrained_features[21]
        self.relu4_3 = vgg_pretrained_features[22]
        self.max_pooling4 = vgg_pretrained_features[23]
        self.conv5_1 = vgg_pretrained_features[24]
        self.relu5_1 = vgg_pretrained_features[25]
        self.conv5_2 = vgg_pretrained_features[26]
        self.relu5_2 = vgg_pretrained_features[27]
        self.conv5_3 = vgg_pretrained_features[28]
        self.relu5_3 = vgg_pretrained_features[29]
        self.max_pooling5 = vgg_pretrained_features[30]
        
        #self.adaptive_pool = vgg16.avgpool
        self.adaptive_pool = nn.AdaptiveAvgPool2d(output_size=(7, 7))
        #self.adaptive_pool = nn.AvgPool2d(kernel_size=1,
        
        self.linear_cl1 = vgg16.classifier[0]
        self.relu_cl1 = vgg16.classifier[1]
        self.linear_cl2 = vgg16.classifier[3]
        self.relu_cl2 = vgg16.classifier[1]
        self.linear_cl3 = vgg16.classifier[6]

        # Turn off these because we'll be using a pretrained network
        # if we didn't do this PyTorch would be saving gradients and eating up precious memory!
        if not requires_grad:
            for param in self.parameters():
                param.requires_grad = False

    # Just expose every single layer during the forward pass
    def forward(self, x):
        x = self.conv1_1(x)
        conv1_1 = x
        x = self.relu1_1(x)
        relu1_1 = x
        x = self.conv1_2(x)
        conv1_2 = x
        x = self.relu1_2(x)
        relu1_2 = x
        x = self.max_pooling1(x)
        x = self.conv2_1(x)
        conv2_1 = x
        x = self.relu2_1(x)
        relu2_1 = x
        x = self.conv2_2(x)
        conv2_2 = x
        x = self.relu2_2(x)
        relu2_2 = x
        x = self.max_pooling2(x)
        x = self.conv3_1(x)
        conv3_1 = x
        x = self.relu3_1(x)
        relu3_1 = x
        x = self.conv3_2(x)
        conv3_2 = x
        x = self.relu3_2(x)
        relu3_2 = x
        x = self.conv3_3(x)
        conv3_3 = x
        x = self.relu3_3(x)
        relu3_3 = x
        x = self.max_pooling3(x)
        x = self.conv4_1(x)
        conv4_1 = x
        x = self.relu4_1(x)
        relu4_1 = x
        x = self.conv4_2(x)
        conv4_2 = x
        x = self.relu4_2(x)
        relu4_2 = x
        x = self.conv4_3(x)
        conv4_3 = x
        x = self.relu4_3(x)
        relu4_3 = x
        x = self.max_pooling4(x)
        x = self.conv5_1(x)
        conv5_1 = x
        x = self.relu5_1(x)
        relu5_1 = x
        x = self.conv5_2(x)
        conv5_2 = x
        x = self.relu5_2(x)
        relu5_2 = x
        x = self.conv5_3(x)
        conv5_3 = x
        x = self.relu5_3(x)
        relu5_3 = x
        x = self.max_pooling5(x)
        mp5 = x
        x = self.adaptive_pool(x)
        ap = x
        x = torch.flatten(x, start_dim=1)
        x = self.linear_cl1(x)
        lin_cl1 = x
        x = self.relu_cl1(x)
        relu_cl1 = x
        x = self.linear_cl2(x)
        lin_cl2 = x
        x = self.relu_cl2(x)
        relu_cl2 = x
        x = self.linear_cl3(x)
        lin_cl3 = x

        # Finally, expose only the layers that you want to experiment with here
        vgg_outputs = namedtuple("VggOutputs", self.layer_names)
        out = vgg_outputs(relu3_3, relu4_1, relu4_2, relu4_3, relu5_1, relu5_2, relu5_3, mp5, ap, relu_cl1, relu_cl2)

        return out
    
vgg_e = Vgg16Experimental(vgg16)

In [20]:
from three_d_shapes_ds import ThreeDShapes
import torchvision
trainloader = torch.utils.data.DataLoader(
                                      ThreeDShapes(filename='data/3dshapes.h5',
                                                   transform=torchvision.transforms.Compose([
                                                       torchvision.transforms.ToPILImage(), 
                                                       torchvision.transforms.Resize((512, 512)),
                                                       torchvision.transforms.ToTensor(),
                                      ]), filtered = False),
                                      batch_size=32, shuffle=True)
for d in trainloader:
    data, target = d[0].cuda(), d[1]
    break

In [4]:
#pip install --upgrade numpy==1.19.2
#!pip install --upgrade torch

In [22]:
dummy_input = Variable(torch.randn(32, 3, 32, 32)).cuda()
print(vgg_e(data).mp5.shape)
print(vgg_e(data).ap.shape)
import matplotlib.pyplot as plt
import numpy as np

print(vgg_e(data).relu_cl2)
#ap = vgg_e(data).ap.detach().cpu()
#for i in range(512):
#    plt.imshow(ap.numpy()[0, i])
#    plt.show()

torch.Size([32, 512, 16, 16])
torch.Size([32, 512, 7, 7])
tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.3368],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.1086],
        [0.0000, 0.0000, 0.4030,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.1746,  ..., 0.0000, 0.0000, 0.2143],
        [0.0000, 0.0000, 0.6016,  ..., 0.0000, 0.0000, 0.2056],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.4375]],
       device='cuda:0')


In [38]:
vgg_e.adaptive_pool

AdaptiveAvgPool2d(output_size=(7, 7))

In [39]:
25088/512

49.0

In [ ]:
from vgg_s import vgg11
vgg_s = 